# Ni(I)–Ph for CO₂ Insertion

Configuration:


In [1]:
from pathlib import Path

# Path to the data file
DATA = Path("projects/ni(I)-co2/descriptors.csv")

# Path to results
RESULTS = Path("projects/ni(I)-co2/results")
RESULTS.mkdir(parents=True, exist_ok=True)

# IDs of the positive and negative reference ligands
POSITIVE = 37  # t-Bu-Xantphos
NEGATIVE = 36  # Xantphos

## Dataset

The dataset is loaded and preprocessed. The `.csv` file contains DFT-descriptors for a variety of ligands and species. Substituent and backbone labels are excluded from the feature matrix, and rows with missing values are removed.

In [2]:
from aixchem.dataset import Dataset

index_column = "ID"
cols_to_drop = ["Substituent", "Backbone"]

data = Dataset(DATA, index=index_column, store_raw=True, sep=";")
data.dropna(axis=0).drop(columns=cols_to_drop)

data.X.head()

,L - HOMO,L - LUMO,L - Q(L),"L2Ni - A(L,Ni,L)","L2Ni - BO(Ni,L)",L2Ni - Bur_V,L2Ni - HOMO,L2Ni - LUMO,L2Ni - Q(Ni),"L2Ni - R(Ni,L)",...,L2NiX2 - LUMO,"L2NiX2 - R(Ni,C)","L2NiX2 - R(Ni,L)","L2NiX2 - Sterimol_B1(Ni,P)","L2NiX2 - Sterimol_B5(Ni,P)","L2NiX2 - Sterimol_L(Ni,P)","L2NiX2 - dBO(Ni,C)",L2NiX2 - dQ(L),L2NiX2 - dQ(Ni),L2NiX2 TO L2Ni + X2
ID,,,,,,,,,,,,,,,,,,,,,
1,-0.20128,0.01769,0.739080,101.226677,0.76940,45.532953,-0.10648,-0.01716,-0.20674,2.107451,...,-0.02883,1.907548,2.195608,2.915291,5.459804,5.693046,0.0610,0.459220,-0.94977,4.410137
2,-0.19569,0.00956,0.741670,101.472221,0.76385,56.829172,-0.10619,-0.02267,-0.19498,2.107843,...,-0.03394,1.906440,2.233532,3.626928,6.309564,6.106117,0.0275,0.440485,-0.91922,0.323795
3,-0.20484,-0.05659,0.782670,100.116513,0.79355,53.756276,-0.11795,-0.05420,-0.09484,2.100174,...,-0.06466,1.904583,2.217178,4.202254,7.880230,7.420312,0.0391,0.407205,-0.90270,-1.797187
4,-0.19388,0.01120,0.754405,104.116952,0.79460,62.827690,-0.10376,-0.02026,-0.18600,2.117131,...,-0.03028,1.900423,2.277310,3.886693,6.817617,6.917496,0.0039,0.410075,-0.86424,-1.516690
5,-0.19817,0.01275,0.735900,101.681301,0.76630,49.224605,-0.10537,-0.02016,-0.20396,2.108726,...,-0.03234,1.908542,2.196318,2.932504,6.218013,6.071036,0.0517,0.474215,-0.96292,6.400597


### Correlation Analysis

To reduce redundancy and multicollinearity, features with a Pearson correlation coefficient above 0.8 are removed. The resulting filtered descriptor set is used in downstream analysis.

In [3]:
from aixchem.transform.preprocess import CorrelationAnalyzer

# Perform correlation analysis and drop highly correlated columns
corr = CorrelationAnalyzer(method="pearson", threshold=0.8)
data = corr.fit(data).transform(data)

# Full correlation matrix
corr.matrix.to_csv(RESULTS / "correlation.csv")

# Correlation matrix after dropping highly correlated (> 0.9) features
# corr.matrix_after.to_csv(RESULTS / "corr_filter.csv")  

### Feature Selection

Features are ranked by their ability to distinguish between two reference ligands:  
- **Positive**: *t*-Bu-Xantphos (Ni(I)–Ph stabilizing)  
- **Negative**: Xantphos (non-stabilizing)  

Features contributing most to this separation are retained using a threshold corresponding approximately to the mean separation across all features.


In [4]:
from aixchem.transform.fselect import FeatureSeparation

fselect = FeatureSeparation()
data = fselect.fit(data, idx=POSITIVE, idy=NEGATIVE).transform(data, threshold=0.2)  # threshold corresponds to approx. mean across all features

# Get ranking of features
fselect.ranking.to_csv(RESULTS / "feature_ranking.csv")

### Feature Scaling

Features are standardized using `StandardScaler` from scikit-learn to ensure comparability in magnitude across the descriptor set prior to PCA and clustering.

In [5]:
from sklearn.preprocessing import StandardScaler
from aixchem.transform.preprocess import Scaler

scaler = Scaler(StandardScaler)
data = scaler.fit(data).transform(data)

## Principal Component Analysis (PCA)

PCA is applied for dimensionality reduction and visualization. The first four principal components are retained. Variance explained by each component and the corresponding feature loadings are exported.

In [6]:
from aixchem.transform.decomposition import PCA

pca = PCA(n_components=4)
pc_data = pca.fit_transform(data)

pca.summary.to_csv(RESULTS / "pca_summary.csv")  # Get summary
pca.loadings.to_csv(RESULTS / "pca_loadings.csv")  # Get eigenvalues

## Clustering Optimization

The optimal number of clusters (*k*) for k-means is determined by evaluating Silhouette and Davies–Bouldin scores across a range of values (2 ≤ *k* ≤ 15). The best-performing *k* is selected for final clustering.

In [7]:
from sklearn.cluster import KMeans
from aixchem.model.cluster import Clusterer
from aixchem.model.optimization import Optimization

# Set optimization parameters 
params = {
    "model": [KMeans],
    "random_state": [42],
    "n_clusters": list(range(2, 16)),
    "n_init": [5000],
    }

opt = Optimization(obj=Clusterer, params=params)

# run in parallel
opt.run(data, njobs=-1)  

# Note: To assess per-sample silhouettes scores at each k, run the optimization sequentially:
# opt.run(data, njobs=1)
# for model in opt.grid:
#     print(model.params, model.silhouettes)

opt.results.to_csv(RESULTS / "optimization.csv")

## Final Clustering (k-means)

K-means clustering is performed using the optimal number of clusters (k = 4). Cluster assignments are recorded alongside the original ligand data and PCA-transformed coordinates. Ligands in the same cluster as the positive reference (*t*-Bu-Xantphos) are identified.

In [8]:
from sklearn.cluster import KMeans
from aixchem.model.cluster import Clusterer

kmeans = Clusterer(KMeans, n_clusters=4, random_state=42, n_init=5000)
kmeans.fit(data)

clusters = kmeans.predict(data)

Create results dataframe:

In [9]:
import pandas as pd

results = pd.concat([data.raw, pc_data.X], axis=1)
results["Cluster"] = clusters

results.to_csv(RESULTS / "results.csv")

# Display ligands that are stored in the cluster of the positive reference:
results.loc[results["Cluster"] == results.loc[POSITIVE]["Cluster"]].index


Index([ 13,  15,  17,  26,  35,  37,  59,  69,  78,  80,  82,  91,  93, 102,
       113, 125, 126, 128, 129, 130, 132],
      dtype='int64', name='ID')

### Cluster Robustness Analysis

Clustering stability is evaluated by rerunning the algorithm 1000 times using different random seeds. For each ligand, a robustness score is calculated based on how frequently it clusters with the positive reference, providing a measure of assignment confidence.


In [10]:
from aixchem.model.cluster import ClusterRobustness

stats = ClusterRobustness(kmeans, random_states=list(range(1001)))
stats.run(data, njobs=-1)

stats_data = stats.check_candidates(POSITIVE)

stats_data.to_csv(RESULTS / "robustness.csv")